# MUSIC GENRE CLASSIFICATION

Install Required Libraries

In [ ]:
!pip3 install catboost
#!pip3 install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.5 MB/s 


Load Libraries

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import re
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,recall_score,precision_score,classification_report

Load Dataset

In [6]:
df_music = pd.read_csv("train.csv")
df_music.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


Dimensionality of the Dataset

In [7]:
df_music.shape

(17996, 17)

Descriptive Statistics of the Attributes

In [8]:
df_music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Artist Name         17996 non-null  object 
 1   Track Name          17996 non-null  object 
 2   Popularity          17568 non-null  float64
 3   danceability        17996 non-null  float64
 4   energy              17996 non-null  float64
 5   key                 15982 non-null  float64
 6   loudness            17996 non-null  float64
 7   mode                17996 non-null  int64  
 8   speechiness         17996 non-null  float64
 9   acousticness        17996 non-null  float64
 10  instrumentalness    13619 non-null  float64
 11  liveness            17996 non-null  float64
 12  valence             17996 non-null  float64
 13  tempo               17996 non-null  float64
 14  duration_in min/ms  17996 non-null  float64
 15  time_signature      17996 non-null  int64  
 16  Clas

In [9]:
df_music.describe()

,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
count,17568.000000,17996.000000,17996.000000,15982.000000,17996.000000,17996.000000,17996.000000,17996.000000,13619.000000,17996.000000,17996.000000,17996.000000,1.799600e+04,17996.000000,17996.000000
mean,44.512124,0.543433,0.662777,5.952447,-7.910660,0.636753,0.079707,0.247082,0.177562,0.196170,0.486208,122.623294,2.007445e+05,3.924039,6.695821
std,17.426928,0.166268,0.235373,3.196854,4.049151,0.480949,0.083576,0.310632,0.304048,0.159212,0.240195,29.571527,1.119891e+05,0.361618,3.206073
min,1.000000,0.059600,0.000020,1.000000,-39.952000,0.000000,0.022500,0.000000,0.000001,0.011900,0.018300,30.557000,5.016500e-01,1.000000,0.000000
25%,33.000000,0.432000,0.509000,3.000000,-9.538000,0.000000,0.034800,0.004300,0.000089,0.097500,0.297000,99.620750,1.663370e+05,4.000000,5.000000
50%,44.000000,0.545000,0.700000,6.000000,-7.016000,1.000000,0.047400,0.081400,0.003910,0.129000,0.481000,120.065500,2.091600e+05,4.000000,8.000000
75%,56.000000,0.659000,0.860000,9.000000,-5.189000,1.000000,0.083000,0.434000,0.200000,0.258000,0.672000,141.969250,2.524900e+05,4.000000,10.000000
max,100.000000,0.989000,1.000000,11.000000,1.355000,1.000000,0.955000,0.996000,0.996000,1.000000,0.986000,217.416000,1.477187e+06,5.000000,10.000000


## Pre-Processing Dataset

In [10]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

df_music['Artist Name'] = df_music['Artist Name'].apply(clean_text)
df_music['Track Name'] = df_music['Track Name'].apply(clean_text)

In [11]:
df_music['Artist Name'].head()

0               bruno mars
1                   boston
2            the raincoats
3                     deno
4    red hot chili peppers
Name: Artist Name, dtype: object

EDA (Exploratory Data Analysis)

In [12]:
df_vis=df_music.copy()

df_submission=pd.read_csv('submission.csv')
df_submission.head()

FileNotFoundError: ignored

In [ ]:
classes=df_submission.columns.tolist()
class_retrieval={retrieve:n for retrieve,n in enumerate(classes,0)}
class_retrieval

{0: 'Acoustic/Folk_0',
 1: 'Alt_Music_1',
 2: 'Blues_2',
 3: 'Bollywood_3',
 4: 'Country_4',
 5: 'HipHop_5',
 6: 'Indie Alt_6',
 7: 'Instrumental_7',
 8: 'Metal_8',
 9: 'Pop_9',
 10: 'Rock_10'}

In [ ]:
#Check the Class Distribution within the Class Attribute
df_vis['Class']=df_music.Class.map(class_retrieval)
df_vis['Class']
#df_vis.Class.value_counts().plot.bar()

0           HipHop_5
1            Rock_10
2        Indie Alt_6
3           HipHop_5
4            Rock_10
            ...     
17991    Indie Alt_6
17992        Blues_2
17993        Metal_8
17994        Metal_8
17995        Rock_10
Name: Class, Length: 17996, dtype: object

Adding Additional Attributes

In [13]:
df_music['Artist_Length'] = df_music['Artist Name'].apply(len) 
df_music['Artist_Char_Count'] = df_music['Artist Name'].str.split().str.len()     
df_music['Track_Length'] = df_music['Track Name'].apply(len)   
df_music['Track_Char_Count'] = df_music['Track Name'].str.split().str.len() 
df_music['Track_Digit'] = df_music['Track Name'].str.findall(r'[0-9]').str.len() 

In [14]:
df_music['Track_in_min']= df_music['duration_in min/ms']/60000
df_music['danceability_by_artist']= df_music.groupby(['Artist Name'])['danceability'].transform('mean')
df_music['energy_by_artist']= df_music.groupby(['Artist Name'])['energy'].transform('mean')
df_music['loudness_by_artist']= df_music.groupby(['Artist Name'])['loudness'].transform('mean')
df_music['acousticness_by_artist']= df_music.groupby(['Artist Name'])['acousticness'].transform('mean')

In [15]:
df_music.sample()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,Artist_Length,Artist_Char_Count,Track_Length,Track_Char_Count,Track_Digit,Track_in_min,danceability_by_artist,energy_by_artist,loudness_by_artist,acousticness_by_artist
1428,the verve,the drugs don work,64.0,0.442,0.52,NaN,-8.648,1,0.0261,0.184,...,9,2,18,4,0,5.081333,0.393333,0.653,-7.675,0.15475


Encoding the Categorical Attributes

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_music['Artist Name'] = le.fit_transform(df_music['Artist Name'])
df_music['Track Name'] = le.fit_transform(df_music['Track Name'])
df_music.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,Artist_Length,Artist_Char_Count,Track_Length,Track_Char_Count,Track_Digit,Track_in_min,danceability_by_artist,energy_by_artist,loudness_by_artist,acousticness_by_artist
0,1146,10929,60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,...,10,2,30,6,0,3.909933,0.658500,0.71125,-4.994750,0.219050
1,1040,4977,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,...,6,1,10,2,0,4.195550,0.407400,0.70140,-7.300600,0.002744
2,7922,9701,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,...,13,2,9,2,0,1.827783,0.493333,0.77400,-8.291333,0.480667
3,1945,6261,66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,...,4,1,17,3,0,2.899467,0.849333,0.66200,-5.996333,0.094667
4,6266,7614,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,...,21,4,28,4,0,3.832667,0.522533,0.86500,-5.837133,0.070362


Dropping the Uneeded Attributes

In [17]:
#df_music.drop(['Artist Name', 'Track Name'], axis=1,inplace=True)
#df_music.drop(['instrumentalness'], axis=1,inplace=True)

Checking the Missing Values

In [18]:
null_values = pd.DataFrame(df_music.isnull().sum(), columns = ['nans']).sort_values('nans', ascending = False)
null_values['null values(%)'] = np.round(df_music.isnull().sum() / len(df_music),3) * 100
null_values

,nans,null values(%)
instrumentalness,4377,24.3
key,2014,11.2
Popularity,428,2.4
Artist Name,0,0.0
time_signature,0,0.0
loudness_by_artist,0,0.0
energy_by_artist,0,0.0
danceability_by_artist,0,0.0
Track_in_min,0,0.0
Track_Digit,0,0.0


## Impute Missing Values

KNN IMPUTER

In [ ]:
'''
knn_impute = KNNImputer(n_neighbors=7)

df_music['instrumentalness']=knn_impute.fit_transform(df_music[['instrumentalness']])
df_music['Popularity']=knn_impute.fit_transform(df_music[['Popularity']])
df_music['key']=knn_impute.fit_transform(df_music[['key']])
'''

"\nknn_impute = KNNImputer(n_neighbors=7)\n\ndf_music['instrumentalness']=knn_impute.fit_transform(df_music[['instrumentalness']])\ndf_music['Popularity']=knn_impute.fit_transform(df_music[['Popularity']])\ndf_music['key']=knn_impute.fit_transform(df_music[['key']])\n"

MEAN

In [19]:
df_music['instrumentalness'].fillna(df_music['instrumentalness'].mean(), inplace = True)
df_music['Popularity'].fillna(df_music['Popularity'].mean(), inplace = True)
df_music['key'].fillna(df_music['key'].mean(), inplace = True)

In [20]:
df_music.isnull().sum()

Artist Name               0
Track Name                0
Popularity                0
danceability              0
energy                    0
key                       0
loudness                  0
mode                      0
speechiness               0
acousticness              0
instrumentalness          0
liveness                  0
valence                   0
tempo                     0
duration_in min/ms        0
time_signature            0
Class                     0
Artist_Length             0
Artist_Char_Count         0
Track_Length              0
Track_Char_Count          0
Track_Digit               0
Track_in_min              0
danceability_by_artist    0
energy_by_artist          0
loudness_by_artist        0
acousticness_by_artist    0
dtype: int64

In [28]:
#Independent
X = df_music.iloc[:,df_music.columns!='Class'].values
#Dependent
y = df_music.iloc[:,df_music.columns=='Class'].values

In [29]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_over, y_over = oversample.fit_resample(X, y)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_over , y_over , test_size=0.2, random_state=42, shuffle=True )
y_train=y_train.ravel()

In [33]:
column_names= ['Artist Name', 'Track Name', 'Popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_in min/ms', 'time_signature',  'Artist_Length',
       'Artist_Char_Count', 'Track_Length', 'Track_Char_Count', 'Track_Digit',
       'Track_in_min', 'danceability_by_artist', 'energy_by_artist',
       'loudness_by_artist', 'acousticness_by_artist']

In [24]:
#X_test = imputer.transform(X_test)
#X_test = pd.DataFrame(X_test, columns= column_names)

In [34]:
# Standard Scaler
def scale_features(X_train, X_test):
    scaler = StandardScaler().fit(X_train)
    Xtrain = scaler.transform(X_train)
    Xtest = scaler.transform(X_test)
    return Xtrain, Xtest

In [35]:
X_train_norm,X_test_norm=scale_features(X_train, X_test)

In [36]:
def Performance_Metrics(y_test,y_pred):
    print('-------------------PERFORMANCE METRICS-------------------')
    print("Accuracy:",100*accuracy_score(y_test, y_pred))
    print('F1 score:', 100*f1_score(y_test, y_pred,average='weighted'))
    print('Recall:', 100*recall_score(y_test, y_pred,average='weighted'))
    print('Precision:', 100*precision_score(y_test, y_pred, average='weighted'))
    print('---------------------------------------------------------')

In [37]:
# Function used for displaying confusion matrix.
def display_confusionMatrix(clf, X, y, title):
  cm = confusion_matrix(y, clf.predict(X), labels=clf.classes_)
  normed_c = (cm.T / cm.astype(np.float).sum(axis=1)).T
  disp = ConfusionMatrixDisplay(confusion_matrix=normed_c,
                                display_labels=clf.classes_)
  disp.plot(
      cmap=plt.cm.Blues
      )
  disp.ax_.set_title(title + " Confusion Matrix:")

In [38]:
import xgboost as xgb
from xgboost import XGBClassifier
xgb_model=xgb.XGBClassifier(random_state=0,use_label_encoder=False,eval_metric='mlogloss')
xgb_model=xgb_model.fit(X_train_norm, y_train)
y_pred=xgb_model.predict(X_test_norm)
cm=confusion_matrix(y_test,y_pred)
Performance_Metrics(y_test,y_pred)

-------------------PERFORMANCE METRICS-------------------
Accuracy: 71.85892725936812
F1 score: 70.86458246225085
Recall: 71.85892725936812
Precision: 70.67041707007049
---------------------------------------------------------


In [39]:
from lightgbm import LGBMClassifier
LG = LGBMClassifier()

LG.fit(X_train_norm, y_train)
lg_pred=xgb_model.predict(X_test_norm)
print(f"accuracy train: {LG.score(X_train_norm, y_train)}")
print(f"accuracy val: {LG.score(X_test_norm, y_test)}")
Performance_Metrics(y_test,lg_pred)

accuracy train: 0.8639066841174715
accuracy val: 0.789033798677443
-------------------PERFORMANCE METRICS-------------------
Accuracy: 71.85892725936812
F1 score: 70.86458246225085
Recall: 71.85892725936812
Precision: 70.67041707007049
---------------------------------------------------------


## MODEL TRAINING & TESTING

In [40]:
# Classifier K-Nearest Neighbors (KN)
classifier_kn = KNeighborsClassifier()
title_kn = "K-Nearest Neighbor"

# LGBM
classifier_lg = LGBMClassifier()
title_lg = "LGBM"

# Random Forest
classifier_rf = RandomForestClassifier()
title_rf = "Random Forest"

# Logistic Regression
classifier_lr = LogisticRegression(max_iter=500,multi_class='multinomial')
title_lr = "Logistic Regression"

# Multi-Layer-Perceptron 
classifier_mlp = MLPClassifier()
title_mlp = "Multi-Layer-Perceptron"

# SVM
classifier_svc = SVC()
title_svc = "Support Vector Machine"

# XGBoost
classifier_xgb=xgb.XGBClassifier(random_state=42,use_label_encoder=False)
title_xgb = "XGBoost"

In [41]:
classifiers = [classifier_lg,classifier_lr, classifier_kn, classifier_mlp, classifier_rf, classifier_svc]

titles = [title_lg,title_lr,title_kn, title_mlp,title_rf, title_svc]

In [42]:
# Fitting training data on-to the classifiers
for clf, title in zip(classifiers, titles):
  clf.fit(X_train_norm, y_train)
  prediction = clf.predict(X_test_norm)
  #Performance_Metrics(y_test,prediction)
  print(title + " Accuracy Score:", round(clf.score(X_test_norm, y_test), 2))

LGBM Accuracy Score: 0.79
Logistic Regression Accuracy Score: 0.65
Decision Tree Accuracy Score: 0.69
K-Nearest Neighbor Accuracy Score: 0.77


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Multi-Layer-Perceptron Accuracy Score: 0.74
Naive Bayes Accuracy Score: 0.6
Random Forest Accuracy Score: 0.8
Support Vector Machine Accuracy Score: 0.75


## HYPERPARAMETER TUNING 

In [43]:
def parameter_tuning(model, hyper_parameter_grid, X_train, y_train ,X_test,y_test):
  # Initialising grid search
  grid = RandomizedSearchCV(estimator=model,
                      param_distributions =  hyper_parameter_grid,
                      scoring='accuracy',
                      verbose=2,
                      cv = 2,
                      n_jobs=-1
                      )
  
  # Inputing the data onto the grid search
  result = grid.fit(X_train, y_train)

  # Displaying the best score with its corrsponding parameters
  print('Best Score: ', result.best_score_)
  print('Best Params: ', result.best_params_)
  print('Best Estimator: ',result.best_estimator_)
  prediction = result.predict(X_test)
  Performance_Metrics(y_test,prediction)

LGBM

In [44]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
lg_param_test ={'num_leaves': sp_randint(2, 40), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [45]:
lg = LGBMClassifier()
parameter_tuning(lg, lg_param_test, X_train_norm, y_train ,X_test_norm,y_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best Score:  0.759362571503529
Best Params:  {'colsample_bytree': 0.9272046994364062, 'min_child_samples': 353, 'min_child_weight': 10.0, 'num_leaves': 35, 'reg_alpha': 0, 'reg_lambda': 0, 'subsample': 0.3002320398422395}
Best Estimator:  LGBMClassifier(colsample_bytree=0.9272046994364062, min_child_samples=353,
               min_child_weight=10.0, num_leaves=35, reg_alpha=0, reg_lambda=0,
               subsample=0.3002320398422395)
-------------------PERFORMANCE METRICS-------------------
Accuracy: 78.17781043350477
F1 score: 77.69444114664334
Recall: 78.17781043350477
Precision: 77.62022124716076
---------------------------------------------------------


SVM

In [ ]:
# defining parameter range
svm_param_grid = {'C': [0.1,1, 10, 100], 
              'gamma': [1,0.1,0.01,0.001],
              'kernel': ['rbf', 'poly', 'sigmoid']}

In [ ]:
svm_model = SVC()
parameter_tuning(svm_model , svm_param_grid, X_train_norm, y_train ,X_test_norm,y_test)

K-Nearest Neighbor

In [46]:
parameters = { 'n_neighbors' : [3,5,7,9,11,13,15],
               'metric' : ['minkowski','euclidean','manhattan']}
neigh = KNeighborsClassifier()
grid_neigh = GridSearchCV(neigh, parameters)
grid_neigh.fit(X_train_norm, y_train) 
Y_pred=grid_neigh.predict(X_test_norm)
accuracy=grid_neigh.score(X_test_norm,y_test)
Performance_Metrics(y_test,Y_pred)

-------------------PERFORMANCE METRICS-------------------
Accuracy: 80.42799412196914
F1 score: 79.31106182695406
Recall: 80.42799412196914
Precision: 80.6565620925726
---------------------------------------------------------


In [ ]:
 print('Best Estimator: ',grid_neigh.best_estimator_)

RANDOM FOREST CLASSIFIER



In [47]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 4, stop = 100, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3,6,7,5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3 , 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
RF_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [48]:
crf= RandomForestClassifier()
parameter_tuning(crf, RF_grid, X_train_norm, y_train ,X_test_norm,y_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best Score:  0.7782369766776809
Best Params:  {'n_estimators': 52, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 90, 'bootstrap': False}
Best Estimator:  RandomForestClassifier(bootstrap=False, max_depth=90, min_samples_split=3,
                       n_estimators=52)
-------------------PERFORMANCE METRICS-------------------
Accuracy: 81.18111682586333
F1 score: 80.73361226465579
Recall: 81.18111682586333
Precision: 80.52211491897769
---------------------------------------------------------


SEQUENTIAL DEEP LEARNING MODELS

In [52]:
import tensorflow as tf
from tensorflow import keras

model_new = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='leaky_relu',input_shape = (X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='leaky_relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='leaky_relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='leaky_relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='leaky_relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='leaky_relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(11,activation = 'softmax')
])


In [53]:
model_new.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [55]:
model_new.fit(X_train_norm, y_train, epochs=100)


Epoch 1/100
1361/1361 [==============================] - 17s 13ms/step - loss: 1.0480 - accuracy: 0.6234
Epoch 2/100
1361/1361 [==============================] - 17s 12ms/step - loss: 0.9565 - accuracy: 0.6526
Epoch 3/100
1361/1361 [==============================] - 16s 12ms/step - loss: 0.9150 - accuracy: 0.6678
Epoch 4/100
1361/1361 [==============================] - 17s 12ms/step - loss: 0.8903 - accuracy: 0.6769
Epoch 5/100
1361/1361 [==============================] - 25s 18ms/step - loss: 0.8650 - accuracy: 0.6830
Epoch 6/100
1361/1361 [==============================] - 19s 14ms/step - loss: 0.8513 - accuracy: 0.6878
Epoch 7/100
1361/1361 [==============================] - 17s 13ms/step - loss: 0.8324 - accuracy: 0.6899
Epoch 8/100
1361/1361 [==============================] - 17s 13ms/step - loss: 0.8232 - accuracy: 0.6959
Epoch 9/100
1361/1361 [==============================] - 18s 13ms/step - loss: 0.8138 - accuracy: 0.7018
Epoch 10/100
1361/1361 [==============================]

In [56]:
test_loss, test_acc = model_new.evaluate(X_test_norm,  y_test, verbose=2)
print('Test accuracy:', test_acc)

341/341 - 1s - loss: 0.5626 - accuracy: 0.7901 - 1s/epoch - 4ms/step
Test accuracy: 0.79013592004776
